In [ ]:
# !pip install splinter
# !pip install selenium
# !pip install webdriver-manager

In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import requests
import pprint
from itertools import chain
import pymongo

In [ ]:
def get_brewery_api():
    '''Reading from the brewery database API'''
    i = 0
    url = f'https://api.openbrewerydb.org/breweries?page={i}&per_page=200'
    brewery_list = []
    for i in range(1,44):
        response = requests.get(url)
        brewery_data = response.json()
        brewery_list.append(brewery_data)
        time.sleep(1)
    all_brewerys_pd = list(chain.from_iterable(brewery_list))
    return all_brewerys_pd

In [ ]:
data_api=get_brewery_api()

In [ ]:
collection_api = db.brewery_api
collection_api.insert_many(data_api)

In [6]:
def scrape_untapped():
    '''Scrape the untapped website to collect the latest beer information
    open the website'''
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=False)
    #reading the html code
    html = browser.html
    soup = BeautifulSoup(html, "lxml")
    
    url_untapped = 'https://untappd.com/brewery/top_rated?country=italy'
    browser.visit(url_untapped)
    time.sleep(1)
    breweries = browser.find_by_xpath('//*[@id="slide"]/div[3]/div/div[1]/div/div[4]')
    
    all_brewery_info = []
    for brewery in breweries:
        title_link = brewery.text
        all_brewery_info.append(title_link)
        time.sleep(2)


    brewery_list = all_brewery_info[0].split("\n")
    
    
    for item in all_brewery_info:
        if item == "Report":
            all_brewery_info.remove("Report Ad")
    
    brewery_list_all = []
    i = 0
    while i < len(brewery_list):
        if i == "Report Ad":
            brewery_list.remove("Report Ad")
        else:
         try:
            brewery = brewery_list[i:i+5]
            #print(brewery)
            dictionary = dict()
            dictionary['brewery_name'] = brewery[0]
            dictionary['brewery_location'] = brewery[1]
            dictionary['brewery_type'] = brewery[2]
            dictionary['number_beers'] = brewery[3]
            dictionary['brewery_ratings'] = brewery[4]
         except IndexError:
            print(f"This batch broke: {brewery}")
            break
        brewery_list_all.append(dictionary)
        i += 5
    browser.quit()
    return brewery_list_all


In [9]:
untappd=scrape_untapped()

In [10]:
untappd

[{'brewery_name': 'Brasseria Della Fonte',
  'brewery_location': 'Pienza, Toscana Italy',
  'brewery_type': 'Micro Brewery',
  'number_beers': '82 Beers 15,020 Ratings',
  'brewery_ratings': '(4.057)'},
 {'brewery_name': "Ca' del Brado",
  'brewery_location': 'Pianoro, Emilia-Romagna Italy',
  'brewery_type': 'Micro Brewery',
  'number_beers': '37 Beers 36,886 Ratings',
  'brewery_ratings': '(3.987)'},
 {'brewery_name': 'Cantina Errante',
  'brewery_location': "Barberino Val d'Elsa, Toscana Italy",
  'brewery_type': 'Nano Brewery',
  'number_beers': '45 Beers 5,227 Ratings',
  'brewery_ratings': '(3.981)'},
 {'brewery_name': 'Alder Beer Co.',
  'brewery_location': 'Seregno, Lombardia Italy',
  'brewery_type': 'Micro Brewery',
  'number_beers': '55 Beers 21,937 Ratings',
  'brewery_ratings': '(3.979)'},
 {'brewery_name': 'Siemàn',
  'brewery_location': 'Villaga, Veneto Italy',
  'brewery_type': 'Micro Brewery',
  'number_beers': '20 Beers 6,172 Ratings',
  'brewery_ratings': '(3.921)'},

In [11]:

# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
#client.drop_database('brewery_db')
# Define database and collection

db = client.brewery_db
collection_untappd = db.brewery_untappd
collection_untappd.insert_many(untappd)

In [24]:
import json

with open('brewery_untappd_export_MD.json', 'r', encoding="utf8") as myfile:
    data_untappd = myfile.read()

data= json.loads(data_untappd)
data

[{'_id': {'$oid': '6423495ebd3839367b2e3bba'},
  'brewery_name': 'Pips Meadery',
  'brewery_location': 'Beach Park, IL United States',
  'brewery_type': 'Meadery',
  'number_beers': '174 Beers 56,661 Ratings',
  'brewery_ratings': '(4.64)'},
 {'_id': {'$oid': '6423495ebd3839367b2e3bbb'},
  'brewery_name': 'Willow Park Brewing',
  'brewery_location': 'Bethlehem, PA United States',
  'brewery_type': 'Nano Brewery',
  'number_beers': '95 Beers 1,774 Ratings',
  'brewery_ratings': '(4.558)'},
 {'_id': {'$oid': '6423495ebd3839367b2e3bbc'},
  'brewery_name': 'Brewlihan',
  'brewery_location': 'Oakland Park, FL United States',
  'brewery_type': 'Meadery',
  'number_beers': '78 Beers 3,974 Ratings',
  'brewery_ratings': '(4.537)'},
 {'_id': {'$oid': '6423495ebd3839367b2e3bbd'},
  'brewery_name': 'Zymarium',
  'brewery_location': 'Orlando, FL United States',
  'brewery_type': 'Meadery',
  'number_beers': '106 Beers 3,935 Ratings',
  'brewery_ratings': '(4.522)'},
 {'_id': {'$oid': '6423495ebd38

In [30]:
data[2]["brewery_name"]


'Brewlihan'

In [90]:
import json

with open('brewery_untappd_export_MD.json', 'r', encoding="utf8") as myfile:
    data_untappd = myfile.read()

data= json.loads(data_untappd)

import re

string = data[3]["brewery_location"]
pattern = "(?!([A-Za-z]+),\s([A-Za-z]+)\s)[A-Za-z]+"

match = re.search(pattern, string)

match

<re.Match object; span=(9, 11), match='FL'>

In [35]:
string

'Orlando, FL United States'

In [37]:
re.find(pattern, string)

AttributeError: module 're' has no attribute 'find'

In [87]:
import re

string = "100 pesos"
pattern = "\d+(?!\d| dollars)"

match = re.search(pattern, string)
match

<re.Match object; span=(0, 3), match='100'>